# Generating music using RBM 

In this mini tutorial we will look into how to generate your own music. I would like to acknowledge Siraj Raval from youtube channel Sirajology for the awesome video.https://www.youtube.com/watch?v=ZE7qWXX05T0 . This is just a help guide for the code example shown by Siraj at his github repo https://github.com/llSourcell/Music_Generator_Demo

Prerequisites for getting the code working is mentioned in github repo of Siraj :  https://github.com/llSourcell/Music_Generator_Demo, however I will state it again for ease :

### Dependencies

* Tensorflow
* pandas
* numpy
* msgpack
* glob
* tqdm

## Understanding Midi
Midi is a python Library used to generate sound from note-patterns. Look the bellow link for knowing more 
http://www.vikparuchuri.com/blog/making-instrumental-music-from-scratch/



In [34]:
import midi
pattern = midi.read_midifile("sample.mid")
print pattern

midi.Pattern(format=1, resolution=220, tracks=\
[midi.Track(\
  [midi.NoteOnEvent(tick=0, channel=0, data=[67, 40]),
   midi.NoteOnEvent(tick=55, channel=0, data=[62, 40]),
   midi.NoteOnEvent(tick=0, channel=0, data=[69, 40]),
   midi.NoteOffEvent(tick=55, channel=0, data=[62, 0]),
   midi.NoteOffEvent(tick=0, channel=0, data=[69, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[59, 40]),
   midi.NoteOnEvent(tick=55, channel=0, data=[29, 40]),
   midi.NoteOnEvent(tick=0, channel=0, data=[62, 40]),
   midi.NoteOffEvent(tick=55, channel=0, data=[29, 0]),
   midi.NoteOffEvent(tick=0, channel=0, data=[59, 0]),
   midi.NoteOffEvent(tick=0, channel=0, data=[62, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[48, 40]),
   midi.NoteOnEvent(tick=0, channel=0, data=[60, 40]),
   midi.NoteOffEvent(tick=55, channel=0, data=[48, 0]),
   midi.NoteOffEvent(tick=0, channel=0, data=[60, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[43, 40]),
   midi.NoteOnEvent(tick=0, channel=0, data=[64, 40])

In [72]:
import midi
# Instantiate a MIDI Pattern (contains a list of tracks)
pattern = midi.Pattern()
# Instantiate a MIDI Track (contains a list of MIDI events)
track = midi.Track()
# Append the track to the pattern
pattern.append(track)
# Instantiate a MIDI note on event, append it to the track
on = midi.NoteOnEvent(tick=0, velocity=20, pitch=midi.G_3)
track.append(on)
# Instantiate a MIDI note off event, append it to the track
off = midi.NoteOffEvent(tick=100, pitch=midi.G_3)
track.append(off)
# Add the end of track event, append it to the track
eot = midi.EndOfTrackEvent(tick=1)
track.append(eot)
# Print out the pattern
print pattern

midi.Pattern(format=1, resolution=220, tracks=\
[midi.Track(\
  [midi.NoteOnEvent(tick=0, channel=0, data=[43, 20]),
   midi.NoteOffEvent(tick=100, channel=0, data=[43, 0]),
   midi.EndOfTrackEvent(tick=1, data=[])])])


## Understanding Serialization format MsgPack
It is a more efficient serialization format for JSON like data. 
![alt text](msgpack.jpg "MsgPack")


In [8]:
import msgpack
packedmsg=msgpack.packb([1, 2, 3])
unpackedmsg=msgpack.unpackb(packedmsg)
print 'packed msg is   : ',packedmsg
print 'unpacked msg is : ',unpackedmsg

packed msg is   :  �
unpacked msg is :  [1, 2, 3]


## Getting Midi Songs in Msgpack serialization

In [17]:
import midi
import msgpack
import midi_manipulation
import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
import numpy as np

def get_songs(path):
    files = glob.glob('{}/*.mid*'.format(path))
    songs = []
    for f in tqdm(files):
        try:
            song = np.array(midi_manipulation.midiToNoteStateMatrix(f))
            if np.array(song).shape[0] > 50:
                songs.append(song)
        except Exception as e:
            raise e           
    return songs

songs = get_songs('Pop_Music_Midi') #These songs have already been converted from midi to msgpack
print "{} songs processed".format(len(songs))

100%|██████████| 126/126 [00:03<00:00, 33.02it/s]

122 songs processed


## Analyzing Processed Songs

In [61]:
for idx,song in enumerate(songs):
    song = np.array(song)
    print song.shape
    if(idx>3):break;

(257, 156)
(129, 156)
(129, 156)
(129, 156)
(257, 156)


The above codes shows the files of variable length encryped into a 156 dimentional vector unit. 156 is nothing but the twice the note range(2*(highest note - lowest note))  
156=2(102-24)

Now we need to use RBM to learn the note pattern.
## RBM Hyperparameters

In [50]:
# Credits : Siraj Raval, Sirajology Youtube Channel
### HyperParameters
# First, let's take a look at the hyperparameters of our model:

lowest_note = midi_manipulation.lowerBound #the index of the lowest note on the piano roll
highest_note = midi_manipulation.upperBound #the index of the highest note on the piano roll
note_range = highest_note-lowest_note #the note range

num_timesteps  = 15 #This is the number of timesteps that we will create at a time
n_visible      = 2*note_range*num_timesteps #This is the size of the visible layer. 
n_hidden       = 50 #This is the size of the hidden layer

num_epochs = 200 #The number of training epochs that we are going to run. For each epoch we go through the entire data set.
batch_size = 100 #The number of training examples that we are going to send through the RBM at a time. 
lr         = tf.constant(0.005, tf.float32) #The learning rate of our model

## RBM Variables

In [60]:
x  = tf.placeholder(tf.float32, [None, n_visible], name="x") #The placeholder variable that holds our data
W  = tf.Variable(tf.random_normal([n_visible, n_hidden], 0.01), name="W") #The weight matrix that stores the edge weights
bh = tf.Variable(tf.zeros([1, n_hidden],  tf.float32, name="bh")) #The bias vector for the hidden layer
bv = tf.Variable(tf.zeros([1, n_visible],  tf.float32, name="bv")) #The bias vector for the visible layer
print 'x  : Shape of Data Variable       : ',x.get_shape()
print 'W  : Shape of Weight Matrix       : ',W.get_shape()
print 'bh : Shape of bias(hidden layer)  : ',bh.get_shape()
print 'bv : Shape of bias(visible layer) : ',bv.get_shape()

x  : Shape of Data Variable       :  (?, 2340)
W  : Shape of Weight Matrix       :  (2340, 50)
bh : Shape of bias(hidden layer)  :  (1, 50)
bv : Shape of bias(visible layer) :  (1, 2340)


## Restricted Boltzman Machine Explaination

The figure bellow shows RBM for the following example. Now What we will do is try to to train this RBM to mimic the existing tunes in an unsupervised fashion. In other words recreated input via hidden units should be as much close to input as possible. There is something called Gibbs energy which comes into play.

$E(x,W)=-(a^Tv+b^Th+v^TWh)$

**E**   :  Gibbs Free Energy  
**a,b** :  bias weights(2340,50)  
**W**   :  Weight Matrix linking hidden and visible units(2340x50)  
**v**   :  visible unit vector(2340x1)  
**h**   :  hidden unit vector(50x1)  

Now this is called Gibbs free Energy which we need to minimize. By minimizing this energy we make sure we get better and better representation for input units, which means network is able to imitate input patterns.
!["Restricted Boltzman Machine"](rbm.png "Restricted Boltzman Machine")

Now what we are going to do, is Manipulate the song data and reshape it such that it becomes 15 timesteps at a time exposed to Visible unit. Again

2340 = 156x15

Now If song was (90x156) will become (6,2340) or each row will contain 15 notes. 

What we do is simple, update the state of Weight Matrix and biases based on recreation of input again.
### Tensor Flow Variable Parameters Update

In [70]:
#This function lets us easily sample from a vector of probabilities
def sample(probs):
    #Takes in a vector of probabilities, and returns a random vector of 0s and 1s sampled from the input vector
    return tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1))

#This function runs the gibbs chain. We will call this function in two places:
#    - When we define the training update step
#    - When we sample our music segments from the trained RBM
def gibbs_sample(k):
    #Runs a k-step gibbs chain to sample from the probability distribution of the RBM defined by W, bh, bv
    def gibbs_step(count, k, xk):
        #Runs a single gibbs step. The visible values are initialized to xk
        hk = sample(tf.sigmoid(tf.matmul(xk, W) + bh)) #Propagate the visible values to sample the hidden values
        xk = sample(tf.sigmoid(tf.matmul(hk, tf.transpose(W)) + bv)) #Propagate the hidden values to sample the visible values
        return count+1, k, xk

    #Run gibbs steps for k iterations
    ct = tf.constant(0) #counter
    [_, _, x_sample] = control_flow_ops.While(lambda count, num_iter, *args: count < num_iter,
                                         gibbs_step, [ct, tf.constant(k), x], 1, False)
    #This is not strictly necessary in this implementation, but if you want to adapt this code to use one of TensorFlow's
    #optimizers, you need this in order to stop tensorflow from propagating gradients back through the gibbs step
    x_sample = tf.stop_gradient(x_sample) 
    return x_sample
### Training Update Code
# Now we implement the contrastive divergence algorithm. First, we get the samples of x and h from the probability distribution
#The sample of x
x_sample = gibbs_sample(1) 
#The sample of the hidden nodes, starting from the visible state of x
h = sample(tf.sigmoid(tf.matmul(x, W) + bh)) 
#The sample of the hidden nodes, starting from the visible state of x_sample
h_sample = sample(tf.sigmoid(tf.matmul(x_sample, W) + bh)) 

#Next, we update the values of W, bh, and bv, based on the difference between the samples that we drew and the original values
size_bt = tf.cast(tf.shape(x)[0], tf.float32)
W_adder  = tf.mul(lr/size_bt, tf.sub(tf.matmul(tf.transpose(x), h), tf.matmul(tf.transpose(x_sample), h_sample)))
bv_adder = tf.mul(lr/size_bt, tf.reduce_sum(tf.sub(x, x_sample), 0, True))
bh_adder = tf.mul(lr/size_bt, tf.reduce_sum(tf.sub(h, h_sample), 0, True))
#When we do sess.run(updt), TensorFlow will run all 3 update steps
updt = [W.assign_add(W_adder), bv.assign_add(bv_adder), bh.assign_add(bh_adder)]

### Tensor Flow Training Session Code

In [71]:
### Run the graph!
# Now it's time to start a session and run the graph! 

with tf.Session() as sess:
    #First, we train the model
    #initialize the variables of the model
    init = tf.initialize_all_variables()
    sess.run(init)
    #Run through all of the training data num_epochs times
    for epoch in tqdm(range(num_epochs)):
        for song in songs:
            #The songs are stored in a time x notes format. The size of each song is timesteps_in_song x 2*note_range
            #Here we reshape the songs so that each training example is a vector with num_timesteps x 2*note_range elements
            song = np.array(song)
            song = song[:np.floor(song.shape[0]/num_timesteps)*num_timesteps]
            song = np.reshape(song, [song.shape[0]/num_timesteps, song.shape[1]*num_timesteps])
            #Train the RBM on batch_size examples at a time
            for i in range(1, len(song), batch_size): 
                tr_x = song[i:i+batch_size]
                sess.run(updt, feed_dict={x: tr_x})

    #Now the model is fully trained, so let's make some music! 
    #Run a gibbs chain where the visible nodes are initialized to 0
    sample = gibbs_sample(1).eval(session=sess, feed_dict={x: np.zeros((10, n_visible))})
    for i in range(sample.shape[0]):
        if not any(sample[i,:]):
            continue
        #Here we reshape the vector to be time x notes, and then save the vector as a midi file
        S = np.reshape(sample[i,:], (num_timesteps, 2*note_range))
        midi_manipulation.noteStateMatrixToMidi(S, "generated_chord_{}".format(i))
 

100%|██████████| 200/200 [01:33<00:00,  2.34it/s]


### Tensorflow Rendering of Midi Notes
Once we have trained The RBM Model, we need to extract synthetic set of Midi Notes. We start the back and fro generative process from zero vector of the size bellow

$dim(xsampling)=10x2340$

or in other words, we are initializing 10 mini musical songs of size 15 (15x156).

That is we are doing gibbs sampling process only once to extract the generated notes and convert it back into midi notes. The RBM will reflect the projection of the weights upon the empty side of visible units. It will go back and forth once as k=1 in *gibbs_sample* function.

Hurrey, so you will see 10 midi files in your folder which are generated via rbm. Try to play them using some midi player and enjoy the awesomeness of machine learning for synthetic music.  
**Special thanks to Siraj Raval via Sirajology for sharing the code**

# Full Code
The full code of all the steps together put together is given bellow :


In [66]:
#This file is heavily based on Daniel Johnson's midi manipulation code in https://github.com/hexahedria/biaxial-rnn-music-composition
# Code by : Siraj Raval,Sirajology
import numpy as np
import pandas as pd
import msgpack
import glob
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
from tqdm import tqdm

###################################################
# In order for this code to work, you need to place this file in the same 
# directory as the midi_manipulation.py file and the Pop_Music_Midi directory

import midi_manipulation

def get_songs(path):
    files = glob.glob('{}/*.mid*'.format(path))
    songs = []
    for f in tqdm(files):
        try:
            song = np.array(midi_manipulation.midiToNoteStateMatrix(f))
            if np.array(song).shape[0] > 50:
                songs.append(song)
        except Exception as e:
            raise e           
    return songs

songs = get_songs('Pop_Music_Midi') #These songs have already been converted from midi to msgpack
print "{} songs processed".format(len(songs))
###################################################

### HyperParameters
# First, let's take a look at the hyperparameters of our model:

lowest_note = midi_manipulation.lowerBound #the index of the lowest note on the piano roll
highest_note = midi_manipulation.upperBound #the index of the highest note on the piano roll
note_range = highest_note-lowest_note #the note range

num_timesteps  = 15 #This is the number of timesteps that we will create at a time
n_visible      = 2*note_range*num_timesteps #This is the size of the visible layer. 
n_hidden       = 50 #This is the size of the hidden layer

num_epochs = 200 #The number of training epochs that we are going to run. For each epoch we go through the entire data set.
batch_size = 100 #The number of training examples that we are going to send through the RBM at a time. 
lr         = tf.constant(0.005, tf.float32) #The learning rate of our model

### Variables:
# Next, let's look at the variables we're going to use:

x  = tf.placeholder(tf.float32, [None, n_visible], name="x") #The placeholder variable that holds our data
W  = tf.Variable(tf.random_normal([n_visible, n_hidden], 0.01), name="W") #The weight matrix that stores the edge weights
bh = tf.Variable(tf.zeros([1, n_hidden],  tf.float32, name="bh")) #The bias vector for the hidden layer
bv = tf.Variable(tf.zeros([1, n_visible],  tf.float32, name="bv")) #The bias vector for the visible layer


#### Helper functions. 

#This function lets us easily sample from a vector of probabilities
def sample(probs):
    #Takes in a vector of probabilities, and returns a random vector of 0s and 1s sampled from the input vector
    return tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1))

#This function runs the gibbs chain. We will call this function in two places:
#    - When we define the training update step
#    - When we sample our music segments from the trained RBM
def gibbs_sample(k):
    #Runs a k-step gibbs chain to sample from the probability distribution of the RBM defined by W, bh, bv
    def gibbs_step(count, k, xk):
        #Runs a single gibbs step. The visible values are initialized to xk
        hk = sample(tf.sigmoid(tf.matmul(xk, W) + bh)) #Propagate the visible values to sample the hidden values
        xk = sample(tf.sigmoid(tf.matmul(hk, tf.transpose(W)) + bv)) #Propagate the hidden values to sample the visible values
        return count+1, k, xk

    #Run gibbs steps for k iterations
    ct = tf.constant(0) #counter
    [_, _, x_sample] = control_flow_ops.While(lambda count, num_iter, *args: count < num_iter,
                                         gibbs_step, [ct, tf.constant(k), x], 1, False)
    #This is not strictly necessary in this implementation, but if you want to adapt this code to use one of TensorFlow's
    #optimizers, you need this in order to stop tensorflow from propagating gradients back through the gibbs step
    x_sample = tf.stop_gradient(x_sample) 
    return x_sample

### Training Update Code
# Now we implement the contrastive divergence algorithm. First, we get the samples of x and h from the probability distribution
#The sample of x
x_sample = gibbs_sample(1) 
#The sample of the hidden nodes, starting from the visible state of x
h = sample(tf.sigmoid(tf.matmul(x, W) + bh)) 
#The sample of the hidden nodes, starting from the visible state of x_sample
h_sample = sample(tf.sigmoid(tf.matmul(x_sample, W) + bh)) 

#Next, we update the values of W, bh, and bv, based on the difference between the samples that we drew and the original values
size_bt = tf.cast(tf.shape(x)[0], tf.float32)
W_adder  = tf.mul(lr/size_bt, tf.sub(tf.matmul(tf.transpose(x), h), tf.matmul(tf.transpose(x_sample), h_sample)))
bv_adder = tf.mul(lr/size_bt, tf.reduce_sum(tf.sub(x, x_sample), 0, True))
bh_adder = tf.mul(lr/size_bt, tf.reduce_sum(tf.sub(h, h_sample), 0, True))
#When we do sess.run(updt), TensorFlow will run all 3 update steps
updt = [W.assign_add(W_adder), bv.assign_add(bv_adder), bh.assign_add(bh_adder)]


### Run the graph!
# Now it's time to start a session and run the graph! 

with tf.Session() as sess:
    #First, we train the model
    #initialize the variables of the model
    init = tf.initialize_all_variables()
    sess.run(init)
    #Run through all of the training data num_epochs times
    for epoch in tqdm(range(num_epochs)):
        for song in songs:
            #The songs are stored in a time x notes format. The size of each song is timesteps_in_song x 2*note_range
            #Here we reshape the songs so that each training example is a vector with num_timesteps x 2*note_range elements
            song = np.array(song)
            song = song[:np.floor(song.shape[0]/num_timesteps)*num_timesteps]
            song = np.reshape(song, [song.shape[0]/num_timesteps, song.shape[1]*num_timesteps])
            #Train the RBM on batch_size examples at a time
            for i in range(1, len(song), batch_size): 
                tr_x = song[i:i+batch_size]
                sess.run(updt, feed_dict={x: tr_x})

    #Now the model is fully trained, so let's make some music! 
    #Run a gibbs chain where the visible nodes are initialized to 0
    sample = gibbs_sample(1).eval(session=sess, feed_dict={x: np.zeros((10, n_visible))})
    for i in range(sample.shape[0]):
        if not any(sample[i,:]):
            continue
        #Here we reshape the vector to be time x notes, and then save the vector as a midi file
        S = np.reshape(sample[i,:], (num_timesteps, 2*note_range))
        midi_manipulation.noteStateMatrixToMidi(S, "generated_chord_{}".format(i))


100%|██████████| 126/126 [00:04<00:00, 31.29it/s]


122 songs processed


100%|██████████| 200/200 [01:30<00:00,  2.24it/s]
